# Day 8: Resonant Collinearity

[https://adventofcode.com/2024/day/8](https://adventofcode.com/2024/day/8)

## Description

### Part One

You find yourselves on the [roof](https://adventofcode.com/2016/day/25) of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar _huge antenna_. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand `<span title="They could have imitated delicious chocolate, but the mediocre chocolate is WAY easier to imitate.">`Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific _frequency_ indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

    ............
    ........0...
    .....0......
    .......0....
    ....0.......
    ......A.....
    ............
    ............
    ........A...
    .........A..
    ............
    ............

The signal only applies its nefarious effect at specific _antinodes_ based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency `a`, they create the two antinodes marked with `#`:

    ..........
    ...#......
    ..........
    ....a.....
    ..........
    .....a....
    ..........
    ......#...
    ..........
    ..........

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

    ..........
    ...#......
    #.........
    ....a.....
    ........a.
    .....a....
    ..#.......
    ......#...
    ..........
    ..........

Antennas with different frequencies don't create antinodes; `A` and `a` count as different frequencies. However, antinodes _can_ occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital `A` creates no antinodes but has a lowercase-`a`\-frequency antinode at its location:

    ..........
    ...#......
    #.........
    ....a.....
    ........a.
    .....a....
    ..#.......
    ......A...
    ..........
    ..........

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost `A`\-frequency antenna:

    ......#....#
    ...#....0...
    ....#0....#.
    ..#....0....
    ....0....#..
    .#....A.....
    ...#........
    #......#....
    ........A...
    .........A..
    ..........#.
    ..........#.

Because the topmost `A`\-frequency antenna overlaps with a `0`\-frequency antinode, there are _`14`_ total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. _How many unique locations within the bounds of the map contain an antinode?_

#### Example

In [252]:
def print_grid(grid: list[list]):
    global rows, cols
    """Prints a 2D grid with borders and aligned axis legends.

    This function takes a 2D grid (list of lists) and prints it to the console with
    borders around the edges and numeric legends for both axes. The grid is formatted with:
    - A top and bottom border made of '+' corners and '-' edges
    - Vertical '|' borders on the left and right sides
    - Single space padding between elements
    - Right-aligned row numbers (y-axis) on the left with fixed width
    - Column numbers (x-axis) on the bottom

    Args:
        grid (List[List]): A 2D grid represented as a list of lists where each inner
        list represents a row of elements.
    """
    # Get dimensions
    rows = len(grid)
    cols = len(grid[0])

    # Calculate width needed for row numbers
    row_num_width = len(str(rows - 1))

    # Create top border
    border = "+" + "-" * (cols * 2 + 1) + "+"

    # Print top border with proper alignment
    print(" " * row_num_width + border)

    # Print each row with fixed-width row number (y-axis)
    for i, row in enumerate(grid):
        print(f"{i:>{row_num_width}}|", end=" ")
        for cell in row:
            print(cell, end=" ")
        print("|")

    # Print bottom border with proper alignment
    print(" " * row_num_width + border)

    # Print column numbers (x-axis) with proper alignment
    print(" " * (row_num_width + 1), end=" ")  # Align with grid
    for j in range(cols):
        print(f"{j:<2}", end="")  # Left-align column numbers with fixed width
    print()

In [253]:
def read_grid(filename: str) -> list[list]:
    """Reads a grid from a file and returns it as a list of lists.

    This function opens a file and processes its content into a grid format, removing any empty lines
    and whitespace from the beginning and end of each line.

    Args:
        filename (str): The path to the file to be read.

    Returns:
        list[list]: A 2D list representing the grid from the file, with each inner list
                    representing a row of the grid.

    Example:
        >>> grid = read_grid("example.txt")
        >>> print(grid)
        [['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9']]
    """
    with open(filename) as f:
        grid = [line.strip() for line in f.read().split("\n") if line.strip()]
    return grid

In [254]:
grid = read_grid("example.txt")
print_grid(grid)

  +-------------------------+
 0| . . . . . . . . . . . . |
 1| . . . . . . . . 0 . . . |
 2| . . . . . 0 . . . . . . |
 3| . . . . . . . 0 . . . . |
 4| . . . . 0 . . . . . . . |
 5| . . . . . . A . . . . . |
 6| . . . . . . . . . . . . |
 7| . . . . . . . . . . . . |
 8| . . . . . . . . A . . . |
 9| . . . . . . . . . A . . |
10| . . . . . . . . . . . . |
11| . . . . . . . . . . . . |
  +-------------------------+
    0 1 2 3 4 5 6 7 8 9 1011


In [255]:
def is_valid_position(y: int, x: int) -> bool:
    """Verify if a position exists within the grid boundaries."""
    return 0 <= y < rows and 0 <= x < cols

In [256]:
import re


def find_antennas(grid: list[list[str]]) -> list[dict[str, tuple[int, int]]]:
    """Finds all alphanumeric characters in a 2D grid and records their positions.

    Args:
        grid (List[List[str]]): A 2D grid represented as a list of lists containing string characters.

    Returns:
        antenna List[Dict[str, Tuple[int, int]]]: A list of dictionaries where each dictionary contains one
        alphanumeric character as the key and its grid coordinates as a tuple value (row, column).

    Example:
        >>> grid = [['*', 'A', '.'],
                    ['.', '1', '*'],
                    ['B', '.', '.']]
        >>> find_antennas(grid)
        [{'A': (0, 1)}, {'1': (1, 1)}, {'B': (2, 0)}]
    """
    antennas = []
    pattern = r"[a-zA-Z0-9]"

    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            match = re.search(pattern, cell)
            if match:
                print(f"Found antenna {match.group()} in position ({i}, {j})")
                antennas.append({match.group(): (i, j)})

    return antennas

In [257]:
antennas = find_antennas(grid)

Found antenna 0 in position (1, 8)
Found antenna 0 in position (2, 5)
Found antenna 0 in position (3, 7)
Found antenna 0 in position (4, 4)
Found antenna A in position (5, 6)
Found antenna A in position (8, 8)
Found antenna A in position (9, 9)


In [258]:
same_freq = []
frequency = "0"
for value in antennas:
    if list(value.keys())[0] == frequency:
        same_freq.append(value)

same_freq

[{'0': (1, 8)}, {'0': (2, 5)}, {'0': (3, 7)}, {'0': (4, 4)}]

In [259]:
def is_inline(
    grid: list[str],
    antenna: dict[str, tuple[int, int]],
    antennas: list[dict[str, tuple[int, int]]],
):
    # Convert grid strings to lists of characters for modification
    grid = [list(row) for row in grid]
    """Check if the antenna is in line with any other antennas.

    Args:
        grid (list[list[str]]): Grid with the location of the antennas.
        antenna dict[str, tuple[int, int]]: Position of the antenna (y, x).
        antennas (list[dict[str, tuple[int, int]]]): List of antennas.
    """

    def define_lines(antennas: list[dict[str, tuple[int, int]]], frequency: str):
        """Generate lines between every two antennas of the same frequency."""
        same_freq = []
        for value in antennas:
            if list(value.keys())[0] == frequency:
                same_freq.append(value)

    def get_radial_line(
        start_pos: tuple[int, int], direction: tuple[int, int]
    ) -> list[tuple[int, int]]:
        # TODO radials are not enough, need to check all positions to see if the is another antenna in line
        """Get all valid positions in a single direction from start position until hitting invalid position.

        Args:
            start_pos (tuple[int, int]): Starting position (y, x)
            direction (tuple[int, int]): Direction to move (dy, dx)

        Returns:
            radial list[tuple[int, int]]: List of valid positions in the radial line
        """
        radial = []
        current_y, current_x = start_pos
        dy, dx = direction

        while True:
            # Calculate next position
            next_y = current_y + dy
            next_x = current_x + dx

            # Check if next position is valid
            if not is_valid_position(next_y, next_x):
                break

            # Add valid position to radial line
            radial.append((next_y, next_x))

            # Update current position
            current_y, current_x = next_y, next_x

        return radial  # TODO return what is the direction of the radial?

    def is_twice_as_far(
        antenna_pos: tuple, antenna_in_line: tuple
    ) -> tuple[tuple[int, int]]:
        """Checks the distance between the two antennas (d) and calculates the
        distance from each antinode to the nearest antenna (d) and to the farthest antenna (2d)."""
        antenna_1 = antenna_pos
        antenna_2 = antenna_in_line

        d = ((antenna_1[0] - antenna_2[0]), (antenna_1[1] - antenna_2[1]))
        print(f"distance between antennas: {d}")

        # TODO distance in both directions of radial are just + and - for one another
        antinode_1 = (antenna_1[0] + d[0], antenna_1[1] + d[1])
        antinode_2 = (antenna_2[0] - d[0], antenna_2[1] - d[1])
        # TODO check if valid position
        print(f"Antinode 1: {antinode_1}, antinode 2: {antinode_2}")

        return antinode_1, antinode_2

    print(f"Checking if antenna {antenna} is in line with another antenna\n")
    antenna_value = list(antenna.keys())[0]
    antenna_pos = antenna[antenna_value]  # !FIXME this position might not be correct
    antennas_in_line = []
    antinode_grid = None
    directions = [
        (-1, -1),  # up-left
        (-1, 0),  # up
        (-1, 1),  # up-right
        (0, -1),  # left
        (0, 1),  # right
        (1, -1),  # down-left
        (1, 0),  # down
        (1, 1),  # down-right
    ]

    for direction in directions:
        radial = get_radial_line(antenna_pos, direction)
        print(f"Radial found: {radial}")
        # TODO return radial to append to radials list?
        for value in antennas:
            if (
                list(value.values())[0] in radial
                and list(value.keys())[0] == antenna_value
            ):
                print(f"Antennas in radial with the same frequency: {value}")
                antennas_in_line.append(list(value.values())[0])
                antinode_1, antinode_2 = is_twice_as_far(antenna_pos, list(value.values())[0])
                if antinode_1:
                    if antinode_grid is None:
                        antinode_grid = [row[:] for row in grid]
                        if is_valid_position(antinode_1[0], antinode_1[1]):
                            antinode_grid[antinode_1[0]][antinode_1[1]] = "#"
                        if antinode_2 and is_valid_position(antinode_2[0], antinode_2[1]):
                            antinode_grid[antinode_2[0]][antinode_2[1]] = "#"
        if antinode_grid:
            print_grid([''.join(row) for row in antinode_grid])
        # print_grid(antinode_grid)
        print()

        # if any(list(antennas.values())[0] in radial for antennas in antennas):
        #     # TODO get value of position of antenna in line
        #     print(f"Antenna {antenna_value} is in line with another antenna")
    print()


is_inline(grid, antennas[-3], antennas)

Checking if antenna {'A': (5, 6)} is in line with another antenna

Radial found: [(4, 5), (3, 4), (2, 3), (1, 2), (0, 1)]

Radial found: [(4, 6), (3, 6), (2, 6), (1, 6), (0, 6)]

Radial found: [(4, 7), (3, 8), (2, 9), (1, 10), (0, 11)]

Radial found: [(5, 5), (5, 4), (5, 3), (5, 2), (5, 1), (5, 0)]

Radial found: [(5, 7), (5, 8), (5, 9), (5, 10), (5, 11)]

Radial found: [(6, 5), (7, 4), (8, 3), (9, 2), (10, 1), (11, 0)]

Radial found: [(6, 6), (7, 6), (8, 6), (9, 6), (10, 6), (11, 6)]

Radial found: [(6, 7), (7, 8), (8, 9), (9, 10), (10, 11)]




#### Claude help (Bresenham line algorithm)

Using itertools

I'll help you create a Python function that will connect all the "0" frequency antennas and store the line points.



This code:

1. Defines a `get_line_points` function that:
   - Takes two points (antenna positions)
   - Uses Bresenham's line algorithm to find all points that lie on the line between them
   - Handles vertical, horizontal, and diagonal lines
   - Returns a list of all points on the line

2. Defines a `connect_antennas` function that:
   - Takes a list of antenna positions
   - Generates all possible pairs of antennas using `itertools.combinations`
   - For each pair, generates all points on the line between them
   - Stores results in a dictionary containing:
     - The antenna pairs
     - The points for each line
     - A set of all unique points across all lines

3. Returns the results in an easily accessible format

The code handles both regular and edge cases (vertical/horizontal lines) and ensures all points between antennas are properly captured.

You can use it like this:
```python
antennas = [
    {'0': (1, 8)},
    {'0': (2, 5)},
    {'0': (3, 7)},
    {'0': (4, 4)}
]

result = connect_antennas(antennas)

# Access the points for a specific line:
first_pair = result['antenna_pairs'][0]
line_points = result['line_points'][first_pair]

# Get all unique points:
all_points = result['all_points']
```

In [260]:
from itertools import combinations


def get_line_points(point1, point2):
    """
    Generate all points that lie on a line between two antenna positions.
    Uses Bresenham's line algorithm adapted for all octants.
    """
    y1, x1 = point1
    y2, x2 = point2
    points = []

    # If line is vertical
    if x1 == x2:
        for y in range(min(y1, y2), max(y1, y2) + 1):
            points.append((y, x1))
        return points

    # If line is horizontal
    if y1 == y2:
        for x in range(min(x1, x2), max(x1, x2) + 1):
            points.append((y1, x))
        return points

    # General case using Bresenham's algorithm
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    steep = dy > dx

    if steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    dx = x2 - x1
    dy = abs(y2 - y1)
    error = dx // 2
    y = y1
    y_step = 1 if y1 < y2 else -1

    for x in range(x1, x2 + 1):
        coord = (y, x) if steep else (x, y)
        points.append(coord)
        error -= dy
        if error < 0:
            y += y_step
            error += dx

    return points


def connect_antennas(antennas):
    """
    Connect all antennas with lines and store the line points.

    Args:
        antennas: List of dictionaries containing antenna positions
                 [{'0': (y1, x1)}, {'0': (y2, x2)}, ...]

    Returns:
        Dictionary containing:
        - antenna_pairs: List of antenna pairs that are connected
        - line_points: Dictionary with antenna pairs as keys and list of points on the line as values
        - all_points: Set of all unique points that make up all lines
    """
    # Extract positions from the antenna dictionaries
    positions = [list(ant.values())[0] for ant in antennas]

    # Generate all possible pairs of antennas
    antenna_pairs = list(combinations(positions, 2))

    # Store results
    result = {"antenna_pairs": antenna_pairs, "line_points": {}, "all_points": set()}

    # Generate points for each line
    for ant1, ant2 in antenna_pairs:
        # Generate key for the line
        line_key = (ant1, ant2)

        # Get all points on the line between these antennas
        line_points = get_line_points(ant1, ant2)

        # Store the line points
        result["line_points"][line_key] = line_points

        # Add points to the set of all points
        result["all_points"].update(line_points)

    return result


# # Example usage:
# antennas = [{"0": (1, 8)}, {"0": (2, 5)}, {"0": (3, 7)}, {"0": (4, 4)}]

# # Generate all lines and points
# result = connect_antennas(antennas)

# # Print results
# print("Antenna pairs:")
# for pair in result["antenna_pairs"]:
#     print(f"  {pair}")
#     print(f"  Line points: {result['line_points'][pair]}\n")

# print(f"Total unique points on all lines: {len(result['all_points'])}")
# print("All points:")
# print(sorted(result["all_points"]))

Without itertools

Yes, I can modify the code to work without the itertools module. We can create our own combination logic using nested loops.

The main change is in the `connect_antennas` function where I replaced the `itertools.combinations` with nested loops. Instead of:
```python
antenna_pairs = list(combinations(positions, 2))
```

We now use:
```python
antenna_pairs = []
for i in range(len(positions)):
    for j in range(i + 1, len(positions)):
        antenna_pairs.append((positions[i], positions[j]))
```

This achieves the same result by:
1. Using an outer loop to select the first antenna (i)
2. Using an inner loop that starts from i+1 to select the second antenna (j)
3. This ensures we get each pair exactly once and don't pair an antenna with itself

In [261]:
def get_line_points(point1, point2):
    """
    Generate all points that lie on a line between two antenna positions.
    Uses Bresenham's line algorithm adapted for all octants.
    """
    y1, x1 = point1
    y2, x2 = point2
    points = []

    # If line is vertical
    if x1 == x2:
        for y in range(min(y1, y2), max(y1, y2) + 1):
            points.append((y, x1))
        return points

    # If line is horizontal
    if y1 == y2:
        for x in range(min(x1, x2), max(x1, x2) + 1):
            points.append((y1, x))
        return points

    # General case using Bresenham's algorithm
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    steep = dy > dx

    if steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    dx = x2 - x1
    dy = abs(y2 - y1)
    error = dx // 2
    y = y1
    y_step = 1 if y1 < y2 else -1

    for x in range(x1, x2 + 1):
        coord = (y, x) if steep else (x, y)
        points.append(coord)
        error -= dy
        if error < 0:
            y += y_step
            error += dx

    return points


def connect_antennas(antennas):
    """
    Connect all antennas with lines and store the line points.

    Args:
        antennas: List of dictionaries containing antenna positions
                 [{'0': (y1, x1)}, {'0': (y2, x2)}, ...]

    Returns:
        Dictionary containing:
        - antenna_pairs: List of antenna pairs that are connected
        - line_points: Dictionary with antenna pairs as keys and list of points on the line as values
        - all_points: Set of all unique points that make up all lines
    """
    # Extract positions from the antenna dictionaries
    positions = [list(ant.values())[0] for ant in antennas]

    # Generate all possible pairs of antennas using nested loops
    antenna_pairs = []
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            antenna_pairs.append((positions[i], positions[j]))

    # Store results
    result = {"antenna_pairs": antenna_pairs, "line_points": {}, "all_points": set()}

    # Generate points for each line
    for ant1, ant2 in antenna_pairs:
        # Generate key for the line
        line_key = (ant1, ant2)

        # Get all points on the line between these antennas
        line_points = get_line_points(ant1, ant2)

        # Store the line points
        result["line_points"][line_key] = line_points

        # Add points to the set of all points
        result["all_points"].update(line_points)

    return result


# # Example usage:
# antennas = [{"0": (1, 8)}, {"0": (2, 5)}, {"0": (3, 7)}, {"0": (4, 4)}]

# # Generate all lines and points
# result = connect_antennas(antennas)

# # Print results
# print("Antenna pairs:")
# for pair in result["antenna_pairs"]:
#     print(f"  {pair}")
#     print(f"  Line points: {result['line_points'][pair]}\n")

# print(f"Total unique points on all lines: {len(result['all_points'])}")
# print("All points:")
# print(sorted(result["all_points"]))

#### Continuation

In [262]:
def create_antinodes(
    grid: list[list[str]],
    antenna_pos: tuple,
    antenna_in_line: tuple,
    line_points: list[tuple, tuple],
    antinode_grid: list[list[str]] = None,
):
    antenna_1 = antenna_pos
    antenna_2 = antenna_in_line

    d = ((antenna_1[0] - antenna_2[0]), (antenna_1[1] - antenna_2[1]))
    print(f"distance between antennas: {d}")

    # TODO distance in both directions of radial are just + and - for one another
    antinode_1 = (antenna_1[0] + d[0], antenna_1[1] + d[1])
    antinode_2 = (antenna_2[0] - d[0], antenna_2[1] - d[1])
    print(f"Antinode 1: {antinode_1}, antinode 2: {antinode_2}")

    # TODO store antinode position even if there is an antenna there
    if antinode_1:
        if antinode_grid is None:
            antinode_grid = [row[:] for row in grid]
            if is_valid_position(antinode_1[0], antinode_1[1]):
                antinode_grid[antinode_1[0]][antinode_1[1]] = "#"
            if antinode_2 and is_valid_position(antinode_2[0], antinode_2[1]):
                antinode_grid[antinode_2[0]][antinode_2[1]] = "#"
                print_grid(["".join(row) for row in antinode_grid])
                return antinode_1, antinode_2
            print_grid(["".join(row) for row in antinode_grid])
            return antinode_1, None
        if antinode_grid:
            print_grid(["".join(row) for row in antinode_grid])
            return antinode_1, antinode_2
    return None, None

In [263]:
# Convert grid strings to lists of characters first
grid_list = [list(row) for row in grid]
antinode_positions = []

for pair in result["antenna_pairs"]:
    # print(f"  {pair}")
    # print(f"  Line points: {result['line_points'][pair]}\n")
    antinode_1, antinode_2 = create_antinodes(grid_list, pair[0], pair[1], result['line_points'][pair])
    if antinode_2 and antinode_1:
        antinode_positions.append(antinode_1)
        antinode_positions.append(antinode_2)
    elif antinode_1:
        antinode_positions.append(antinode_1)

distance between antennas: (-1, 3)
Antinode 1: (0, 11), antinode 2: (3, 2)
  +-------------------------+
 0| . . . . . . . . . . . # |
 1| . . . . . . . . 0 . . . |
 2| . . . . . 0 . . . . . . |
 3| . . # . . . . 0 . . . . |
 4| . . . . 0 . . . . . . . |
 5| . . . . . . A . . . . . |
 6| . . . . . . . . . . . . |
 7| . . . . . . . . . . . . |
 8| . . . . . . . . A . . . |
 9| . . . . . . . . . A . . |
10| . . . . . . . . . . . . |
11| . . . . . . . . . . . . |
  +-------------------------+
    0 1 2 3 4 5 6 7 8 9 1011
distance between antennas: (-2, 1)
Antinode 1: (-1, 9), antinode 2: (5, 6)
  +-------------------------+
 0| . . . . . . . . . . . . |
 1| . . . . . . . . 0 . . . |
 2| . . . . . 0 . . . . . . |
 3| . . . . . . . 0 . . . . |
 4| . . . . 0 . . . . . . . |
 5| . . . . . . # . . . . . |
 6| . . . . . . . . . . . . |
 7| . . . . . . . . . . . . |
 8| . . . . . . . . A . . . |
 9| . . . . . . . . . A . . |
10| . . . . . . . . . . . . |
11| . . . . . . . . . . . . |
  +--------

The error occurs because we're trying to modify strings in the grid, but strings are immutable in Python. We need to convert the grid of strings into a grid of lists first. Here's the fix:



Made changes.

In [264]:
antinode_positions

[(0, 11),
 (3, 2),
 (-1, 9),
 (5, 6),
 (-2, 12),
 (7, 0),
 (-3, 10),
 (9, 4),
 (-6, 8),
 (-7, 7),
 (1, 3),
 (4, 9),
 (0, 6),
 (6, 3),
 (-1, 4),
 (8, 7),
 (-4, 2),
 (-5, 1),
 (2, 10),
 (5, 1),
 (1, 8),
 (7, 5),
 (-2, 6),
 (-3, 5),
 (3, 2),
 (6, 8),
 (0, 0),
 (-1, -1),
 (2, 4),
 (11, 10),
 (1, 3),
 (7, 7),
 (10, 10)]

#### Running the example

In [269]:
def create_antinodes(
    grid: list[list[str]],
    antenna_pos: tuple,
    antenna_in_line: tuple,
    antinode_grid: list[list[str]] = None,
) -> tuple[tuple[int, int] | None, tuple[int, int] | None]:
    """
    Create antinodes for a pair of antennas.

    Args:
        grid: The grid as a list of lists
        antenna_pos: Position of the first antenna (y, x)
        antenna_in_line: Position of the second antenna (y, x)
        antinode_grid: Optional grid to display antinodes

    Returns:
        Tuple of two antinode positions. Each position can be None if invalid.
    """
    # Calculate the vector between antennas
    d = (antenna_in_line[0] - antenna_pos[0], antenna_in_line[1] - antenna_pos[1])

    # Calculate potential antinode positions
    antinode_1 = (antenna_pos[0] - d[0], antenna_pos[1] - d[1])
    antinode_2 = (antenna_in_line[0] + d[0], antenna_in_line[1] + d[1])

    # Initialize result antinodes
    valid_antinode_1 = antinode_1 if is_valid_position(antinode_1[0], antinode_1[1]) else None
    valid_antinode_2 = antinode_2 if is_valid_position(antinode_2[0], antinode_2[1]) else None

    # If display grid is requested
    if antinode_grid is None:
        antinode_grid = [row[:] for row in grid]
        if valid_antinode_1:
            antinode_grid[valid_antinode_1[0]][valid_antinode_1[1]] = "#"
        if valid_antinode_2:
            antinode_grid[valid_antinode_2[0]][valid_antinode_2[1]] = "#"
        print("\nAntinodes created at:", valid_antinode_1, valid_antinode_2)
        # print_grid(["".join(row) for row in antinode_grid])

    return valid_antinode_1, valid_antinode_2

In [270]:
# Convert grid strings to lists of characters first
grid_list = [list(row) for row in grid]
antinode_positions = []
result = connect_antennas(antennas)

# TODO lines should only happen between antennas of the same frequency
for pair in result["antenna_pairs"]:
    antinode_1, antinode_2 = create_antinodes(grid_list, pair[0], pair[1])
    if antinode_1:
        antinode_positions.append(antinode_1)
    if antinode_2:
        antinode_positions.append(antinode_2)

print()
print(f'Number of antinodes: {len(antinode_positions)}')
print(f'Number of unique antinode positions: {len(set(antinode_positions))}')


Antinodes created at: (0, 11) (3, 2)

Antinodes created at: None (5, 6)

Antinodes created at: None (7, 0)

Antinodes created at: None (9, 4)

Antinodes created at: None None

Antinodes created at: None None

Antinodes created at: (1, 3) (4, 9)

Antinodes created at: (0, 6) (6, 3)

Antinodes created at: None (8, 7)

Antinodes created at: None None

Antinodes created at: None None

Antinodes created at: (2, 10) (5, 1)

Antinodes created at: (1, 8) (7, 5)

Antinodes created at: None None

Antinodes created at: None None

Antinodes created at: (3, 2) (6, 8)

Antinodes created at: (0, 0) None

Antinodes created at: None None

Antinodes created at: (2, 4) (11, 10)

Antinodes created at: (1, 3) None

Antinodes created at: (7, 7) (10, 10)

Number of antinodes: 22
Number of unique antinode positions: 20


In [267]:
# Initialize permanent grid once at start
grid_list = [list(row) for row in grid]
antinode_positions = set()
permanent_antinode_grid = [list(row) for row in grid]  # Create fresh grid copy


# Modify create_antinodes to always use and update permanent grid
def create_antinodes(
    grid: list[list[str]],
    antenna_pos: tuple,
    antenna_in_line: tuple,
    antinode_grid: list[list[str]],
) -> tuple[tuple[int, int] | None, tuple[int, int] | None]:
    d = (antenna_in_line[0] - antenna_pos[0], antenna_in_line[1] - antenna_pos[1])

    antinode_1 = (antenna_pos[0] - d[0], antenna_pos[1] - d[1])
    antinode_2 = (antenna_in_line[0] + d[0], antenna_in_line[1] + d[1])

    valid_antinode_1 = (
        antinode_1 if is_valid_position(antinode_1[0], antinode_1[1]) else None
    )
    valid_antinode_2 = (
        antinode_2 if is_valid_position(antinode_2[0], antinode_2[1]) else None
    )

    # Always mark antinodes in the permanent grid
    if valid_antinode_1:
        antinode_grid[valid_antinode_1[0]][valid_antinode_1[1]] = "#"
    if valid_antinode_2:
        antinode_grid[valid_antinode_2[0]][valid_antinode_2[1]] = "#"

    return valid_antinode_1, valid_antinode_2


# Process antenna pairs
result = connect_antennas(antennas)
for pair in result["antenna_pairs"]:
    antinode_1, antinode_2 = create_antinodes(
        grid_list, pair[0], pair[1], permanent_antinode_grid
    )
    if antinode_1:
        antinode_positions.add(antinode_1)
    if antinode_2:
        antinode_positions.add(antinode_2)

# Print final result
print("\nFinal grid with all antinodes:")
print_grid(["".join(row) for row in permanent_antinode_grid])
print(f"Number of unique antinode positions: {len(antinode_positions)}")


Final grid with all antinodes:
  +-------------------------+
 0| # . . . . . # . . . . # |
 1| . . . # . . . . # . . . |
 2| . . . . # 0 . . . . # . |
 3| . . # . . . . 0 . . . . |
 4| . . . . 0 . . . . # . . |
 5| . # . . . . # . . . . . |
 6| . . . # . . . . # . . . |
 7| # . . . . # . # . . . . |
 8| . . . . . . . # A . . . |
 9| . . . . # . . . . A . . |
10| . . . . . . . . . . # . |
11| . . . . . . . . . . # . |
  +-------------------------+
    0 1 2 3 4 5 6 7 8 9 1011
Number of unique antinode positions: 20


In [271]:
def get_line_points(point1, point2):
    """
    Generate all points that lie on a line between two antenna positions.
    Uses Bresenham's line algorithm adapted for all octants.
    """
    y1, x1 = point1
    y2, x2 = point2
    points = []

    # If line is vertical
    if x1 == x2:
        for y in range(min(y1, y2), max(y1, y2) + 1):
            points.append((y, x1))
        return points

    # If line is horizontal
    if y1 == y2:
        for x in range(min(x1, x2), max(x1, x2) + 1):
            points.append((y1, x))
        return points

    # General case using Bresenham's algorithm
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    steep = dy > dx

    if steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    dx = x2 - x1
    dy = abs(y2 - y1)
    error = dx // 2
    y = y1
    y_step = 1 if y1 < y2 else -1

    for x in range(x1, x2 + 1):
        coord = (y, x) if steep else (x, y)
        points.append(coord)
        error -= dy
        if error < 0:
            y += y_step
            error += dx

    return points


def connect_frequency_antennas(antennas):
    """
    Connect antennas with lines only if they share the same frequency.

    Args:
        antennas: List of dictionaries containing antenna positions and frequencies
                 [{'0': (y1, x1)}, {'0': (y2, x2)}, {'A': (y3, x3)}, ...]

    Returns:
        Dictionary containing:
        - antenna_pairs: List of antenna pairs that are connected, with their frequency
        - line_points: Dictionary with antenna pairs as keys and list of points on the line as values
        - all_points: Set of all unique points that make up all lines
    """
    # Group antennas by frequency
    frequency_groups = {}
    for antenna in antennas:
        for freq, pos in antenna.items():
            if freq not in frequency_groups:
                frequency_groups[freq] = []
            frequency_groups[freq].append(pos)

    # Store results
    result = {
        "antenna_pairs": [],
        "line_points": {},
        "all_points": set(),
        "frequency_lines": {},  # Store lines by frequency
    }

    # Process each frequency group separately
    for freq, positions in frequency_groups.items():
        result["frequency_lines"][freq] = []

        # Generate pairs for this frequency
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                ant1, ant2 = positions[i], positions[j]
                pair = (ant1, ant2)

                # Store the pair with its frequency
                result["antenna_pairs"].append((freq, pair))

                # Generate line points
                line_points = get_line_points(ant1, ant2)

                # Store the line points
                result["line_points"][pair] = line_points
                result["frequency_lines"][freq].append(line_points)

                # Add to all points
                result["all_points"].update(line_points)

    return result

# Generate all lines and points
# TODO save only unique antinodes
result = connect_frequency_antennas(antennas)

# Print results
print("Lines by frequency:")
for freq, pairs in result["frequency_lines"].items():
    print(f"\nFrequency '{freq}':")
    for line_points in pairs:
        print(f"  Line points: {line_points}")

print("\nTotal unique points on all lines:", len(result["all_points"]))
print("\nAntenna pairs with frequencies:")
for freq, pair in result["antenna_pairs"]:
    print(f"  Frequency '{freq}': {pair}")
    print(f"  Line points: {result['line_points'][pair]}\n")

Lines by frequency:

Frequency '0':
  Line points: [(5, 2), (6, 2), (7, 1), (8, 1)]
  Line points: [(8, 1), (8, 2), (7, 3)]
  Line points: [(4, 4), (5, 3), (6, 3), (7, 2), (8, 1)]
  Line points: [(5, 2), (6, 2), (7, 3)]
  Line points: [(5, 2), (5, 3), (4, 4)]
  Line points: [(4, 4), (5, 4), (6, 3), (7, 3)]

Frequency 'A':
  Line points: [(6, 5), (7, 6), (7, 7), (8, 8)]
  Line points: [(6, 5), (7, 6), (7, 7), (8, 8), (9, 9)]
  Line points: [(8, 8), (9, 9)]

Total unique points on all lines: 16

Antenna pairs with frequencies:
  Frequency '0': ((1, 8), (2, 5))
  Line points: [(5, 2), (6, 2), (7, 1), (8, 1)]

  Frequency '0': ((1, 8), (3, 7))
  Line points: [(8, 1), (8, 2), (7, 3)]

  Frequency '0': ((1, 8), (4, 4))
  Line points: [(4, 4), (5, 3), (6, 3), (7, 2), (8, 1)]

  Frequency '0': ((2, 5), (3, 7))
  Line points: [(5, 2), (6, 2), (7, 3)]

  Frequency '0': ((2, 5), (4, 4))
  Line points: [(5, 2), (5, 3), (4, 4)]

  Frequency '0': ((3, 7), (4, 4))
  Line points: [(4, 4), (5, 4), (6, 

In [272]:
# Initialize permanent grid once at start
grid_list = [list(row) for row in grid]
antinode_positions = set()
permanent_antinode_grid = [list(row) for row in grid]  # Create fresh grid copy


# Modify create_antinodes to always use and update permanent grid
def create_antinodes(
    grid: list[list[str]],
    antenna_pos: tuple,
    antenna_in_line: tuple,
    antinode_grid: list[list[str]],
) -> tuple[tuple[int, int] | None, tuple[int, int] | None]:
    d = (antenna_in_line[0] - antenna_pos[0], antenna_in_line[1] - antenna_pos[1])

    antinode_1 = (antenna_pos[0] - d[0], antenna_pos[1] - d[1])
    antinode_2 = (antenna_in_line[0] + d[0], antenna_in_line[1] + d[1])

    valid_antinode_1 = (
        antinode_1 if is_valid_position(antinode_1[0], antinode_1[1]) else None
    )
    valid_antinode_2 = (
        antinode_2 if is_valid_position(antinode_2[0], antinode_2[1]) else None
    )

    # Always mark antinodes in the permanent grid
    if valid_antinode_1:
        antinode_grid[valid_antinode_1[0]][valid_antinode_1[1]] = "#"
    if valid_antinode_2:
        antinode_grid[valid_antinode_2[0]][valid_antinode_2[1]] = "#"

    return valid_antinode_1, valid_antinode_2


# Process antenna pairs
result = connect_antennas(antennas)
for pair in result["antenna_pairs"]:
    antinode_1, antinode_2 = create_antinodes(
        grid_list, pair[0], pair[1], permanent_antinode_grid
    )
    if antinode_1:
        antinode_positions.add(antinode_1)
    if antinode_2:
        antinode_positions.add(antinode_2)

# Print final result
print("\nFinal grid with all antinodes:")
print_grid(["".join(row) for row in permanent_antinode_grid])
print(f"Number of unique antinode positions: {len(antinode_positions)}")


Final grid with all antinodes:
  +-------------------------+
 0| # . . . . . # . . . . # |
 1| . . . # . . . . # . . . |
 2| . . . . # 0 . . . . # . |
 3| . . # . . . . 0 . . . . |
 4| . . . . 0 . . . . # . . |
 5| . # . . . . # . . . . . |
 6| . . . # . . . . # . . . |
 7| # . . . . # . # . . . . |
 8| . . . . . . . # A . . . |
 9| . . . . # . . . . A . . |
10| . . . . . . . . . . # . |
11| . . . . . . . . . . # . |
  +-------------------------+
    0 1 2 3 4 5 6 7 8 9 1011
Number of unique antinode positions: 20
